In [38]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()


CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")
AZURE_AI_SEARCH_ENDPOINT= os.environ['AZURE_AI_SEARCH_ENDPOINT']
AZURE_AI_SEARCH_API_KEY= os.environ['AZURE_AI_SEARCH_API_KEY']
AZURE_OPENAI_ENDPOINT= os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_API_KEY= os.environ['AZURE_API_KEY']
AZURE_OPENAI_LLM_API_KEY= os.environ['AZURE_OPENAI_LLM_API_KEY']
AZURE_OPENAI_LLM_ENDPOINT= os.environ['AZURE_OPENAI_LLM_ENDPOINT']

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore
from llama_index.vector_stores.azureaisearch import IndexManagement
from llama_index.core import StorageContext
from azure.core.credentials import AzureKeyCredential
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding


# Init and loading index from vector store

#### Init embed model (using my own embed model hosted on azure AI foundry)

In [33]:
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",
    deployment_name="text-embedding-3-small",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_API_KEY,
    api_version="2024-02-01", 
)

#### Init index_client 
`(using my own index (vector db - Azure AI search) hosted on azure AI foundry)`

In [34]:
credential = AzureKeyCredential(AZURE_AI_SEARCH_API_KEY)
index_client = SearchIndexClient(endpoint=AZURE_AI_SEARCH_ENDPOINT, credential=credential)

#### Init Vector Store

In [35]:
index_name = 'it_can_be_done'

vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    embed_model=embed_model,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
)

async_search_or_index_client is None. Depending on the client type passed in, sync or async functions may not work.


#### Providind vector_store as storage_context

In [36]:
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    )

#### Loading index from stored vectors

In [37]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, 
    embed_model=embed_model,
    storage_context=storage_context
)

<h5 style="color:green">Index Loaded successfully. [READY FOR QUERYING]</h5>

### Query Engine

* init LLM

In [40]:
from llama_index.llms.azure_openai import AzureOpenAI

llm = AzureOpenAI(
    model='gpt-4.1-mini',
    api_key=AZURE_OPENAI_LLM_API_KEY,
    azure_endpoint=AZURE_OPENAI_LLM_ENDPOINT,
    api_version='2024-02-01',
    engine='gpt-4.1-mini'
)

* querying the LLM

In [41]:
query_engine = index.as_query_engine(llm=llm, streaming=True)
response = query_engine.query(
    'How do I go rich?'
)

response.print_response_stream()

To become rich, focus on building wealth rather than simply accumulating money or seeking status. Wealth comes from owning assets that generate income even while you sleep. It's important to believe that creating wealth ethically is possible, and to avoid resenting wealth, as that mindset can prevent you from achieving it. Ignore those who play status games and criticize others for pursuing wealth. Instead of trading your time for money, aim to own equity in a business, as this is key to financial freedom. Ultimately, you will become rich by providing something valuable to society that people want but don't yet know how to obtain, and doing so on a large scale.

* chat engine

In [46]:
from llama_index.core.chat_engine.types import ChatMode

chat_engine = index.as_chat_engine(llm=llm, chat_mode=ChatMode.BEST)

chat_engine.chat_repl()

/Users/atulgupta/workspace/rag-handson/venv/lib/python3.13/site-packages/llama_index/agent/openai/base.py:144: DeprecationWarning: Call to deprecated class OpenAIAgent. (OpenAIAgent has been deprecated and is not maintained.

`FunctionAgent` is the recommended replacement.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
/Users/atulgupta/workspace/rag-handson/venv/lib/python3.13/site-packages/deprecated/classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)
/Users/atulgupta/workspace/rag-handson/venv/lib/python3.13/site-packages/llama_index/agent/openai/step.py:213: DeprecationWarning: Call to deprecated class OpenAIAgentWork

===== Entering Chat REPL =====
Type "exit" to exit.



* using reACT mode

In [47]:
react_chat_engine = index.as_chat_engine(llm=llm, chat_mode=ChatMode.REACT)
react_chat_engine.streaming_chat_repl()

/Users/atulgupta/workspace/rag-handson/venv/lib/python3.13/site-packages/llama_index/agent/openai/base.py:144: DeprecationWarning: Call to deprecated class OpenAIAgent. (OpenAIAgent has been deprecated and is not maintained.

`FunctionAgent` is the recommended replacement.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
/Users/atulgupta/workspace/rag-handson/venv/lib/python3.13/site-packages/deprecated/classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)
/Users/atulgupta/workspace/rag-handson/venv/lib/python3.13/site-packages/llama_index/agent/openai/step.py:213: DeprecationWarning: Call to deprecated class OpenAIAgentWork

===== Entering Chat REPL =====
Type "exit" to exit.

Assistant: Staying happy and bright all the time is a wonderful goal, but it’s important to remember that everyone experiences ups and downs. However, there are habits and mindsets that can help you maintain a more positive and resilient outlook:

### 1. **Practice Gratitude**
- Regularly reflect on things you’re thankful for. Keeping a gratitude journal can help shift your focus to the positive aspects of your life.

### 2. **Take Care of Your Body**
- Exercise regularly, eat nutritious foods, and get enough sleep. Physical health has a big impact on mood and energy.

### 3. **Connect with Others**
- Spend time with friends and family. Social connections are key to happiness.

### 4. **Engage in Activities You Enjoy**
- Make time for hobbies and activities that bring you joy and fulfillment.

### 5. **Practice Mindfulness and Meditation**
- Mindfulness helps you stay present and reduces stress. Meditation can improve your overall se

* Chat Memory Buffer

In [48]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer(token_limit=1500)

In [52]:
chat_engine = index.as_chat_engine(
    llm=llm,
    chat_mode='context',
    memory=memory,
    system_prompt= (
        'You are a wise librarian who knows about a lot of books, but your favourite one is Almanack of Naval Ravikant, and you can answer anything asked about this books or its teachings.'
    )
)

response = chat_engine.streaming_chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Assistant: That’s a wonderful and important question—one that Naval Ravikant addresses with great insight in *The Almanack of Naval Ravikant*.

**Naval’s Perspective:**

On page 90, Naval explains that making money can solve your money problems, but it won’t automatically make you happy or stress-free. In fact, the process of becoming rich often requires you to become a “high-anxiety, high-stress, hard-working, competitive person.” If you spend decades in that mode, it becomes your default state, and even after achieving wealth, you may find it difficult to turn off the stress and anxiety.

**Key Lessons from Naval:**

1. **Money Solves Money Problems, Not Happiness:**  
   Money removes obstacles related to financial insecurity, but happiness is a separate pursuit. Many wealthy people remain unhappy because they haven’t learned how to be happy—they’ve only learned how to be successful in a material sense.

2. **Train Yourself for Ha

### Customizing Querying

In [59]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

1- Configure a Retriever

In [61]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10
)

2- Configure a postprocessor

In [57]:
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.5)

3- Configure a response synthesizer

In [60]:
response_synthesizer = get_response_synthesizer(llm=llm)

4- Create a Query Engine

In [62]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[postprocessor],
)

In [70]:
response = query_engine.query('Staying honest yet acheiving excellence and success, how is it made possible?')

In [72]:
print(response)

Achieving excellence and success while staying honest is possible by integrating honesty into your approach to life and decision-making. Honesty is not about being blunt or negative, but about being authentic and congruent—saying what you think and thinking what you say. This alignment frees you from self-deception and keeps you connected to reality, which is essential for sound judgment and effective decisions.

When offering criticism, focus on critiquing general approaches or classes of activities rather than individuals, and when praising, be specific and direct with individuals. This method helps avoid triggering defensiveness and instead enlists people’s egos to work with you, not against you.

Charisma can be developed by projecting both confidence and love, making it possible to be honest while remaining positive. Excellence in decision-making comes from discarding biases, memory, and identity when facing important choices, and focusing solely on the problem at hand. This clari